# 挑战任务五：递推数列的隐藏规律
 
**学习目标**：体验机器学习训练过程，感受从数学问题到编程解决的完整流程


## 第一部分：破译思路

### 1.1 数据准备思路📂

为了**训练**计算机，我们需要准备"练习题"：

从数列 $a_{1},a_{2},a_{3},a_{4},a_{5},a_{6},\cdots$ 中，我们可以构造多个**训练样本**：

- 样本1：用 $a_{1},a_{2}$ 预测 **$a_{4}$**
- 样本2：用 $a_{2},a_{3}$ 预测 **$a_{5}$**
- 样本3：用 $a_{3},a_{4}$ 预测 **$a_{4}$**

**为什么这样构造？**  
因为这与我们的目标关系 $a_{n + 3} = p \bullet a_{n + 1} - q \bullet a_{n}$ 完全对应！

**✅ 填空（请填写在纸质材料上）：**  
我们构造**训练样本**时，用 _______ 和 _______ 作为输入，用 _______ 作为预测目标。



### 1.2 从函数到机器学习

现在我们已经有了清晰的函数关系：

$$\text{输出} = p \bullet y - q \bullet x$$

**机器学习的思路：**  
我们不求一次就找到完美的 $p$ 和 $q$，而是让计算机"学习"：

**举个例子：**  
假设真实规律是 $a_{n + 3} = 2 \bullet a_{n + 1} - 1 \bullet a_{n}$，其中 $p =$ 2，$q =$ 1

我们有一个数据样本：

- 输入：$a_{1} = 3,a_{2} = 5$
- 真实目标值：$a_{4} = 2 \times 5 - 1 \times 3 =$ _______

计算机的学习过程：

- 第1次尝试：$p = 3.1,q = 2.0$  
  输出预测值：$a_{4}$ = $3.1 \times 5 - 2.0 \times 3 =$ _______，误差 = _______
- 第2次尝试：$p = 2.9,q = 1.7$  
  输出预测值：$a_{4}$  = $2.9 \times 5 - 1.7 \times 3 = 9.4$，误差 = $\mid 7 - 9.4 \mid = 2.4$
- ...
- 第10000次尝试：$p = 2.01,q = 1.02$  
  输出预测值：$a_{4}$  = $2.01 \times 5 - 1.02 \times 3 = 6.99$，误差 = $\mid 7 - 6.99 \mid = 0.01$

**重要说明：** 计算机的尝试不是盲目的，它会根据误差的大小和方向，智能地调整 $p$ 和 $q$，让它们朝着误差越来越小的方向变化。


**✅ 填空（请填写在纸质材料上）：**  

机器学习通过多次 _______ 和 _______，让预测值从"差得很远"逐步变得"非常接近"真实值，这就是机器学习的**训练**过程

通过这个例子可以看到，机器学习让预测误差从最初的 _______ 降到了最终的 _______。

**手动调参：** 

在机器学习中，计算机通过智能算法自动调整参数，作为对比，我们还可以体验一下最基础的“随机尝试法”。

下面这个程序模拟了随机尝试的过程：你需要手动输入 p 和 q 的值，通过观察误差来不断调整两个参数。这能让你体会到：如果没有智能算法，寻找最优参数是多么困难！准备好了吗？开始你的“手动调参”之旅吧！

In [ ]:
# 动手练习1，请运行下方代码，输入p,q进行尝试

print("=== 找规律游戏 ===")
print("a1=8, a2=4, a4=-31.6")
print("公式: a4 = p×a2 - q×a1")

while True:
    try:
        p, q = map(float, input("\n输入 p q(用空格隔开)，输入0 0退出: ").split())
        result = p * 4 - q * 8
        error = abs(-31.6 - result)
        print(f"结果: {result:.1f}, 误差: {error:.1f}")

        if error < 0.1:
            print("正确! p=3.1, q=5.5")
            break
        elif p==0 and q==0:
            print('已退出')
            break
    except:
        break

## 第二部分：编程实践 - 数据准备与处理

数学理论成立，我们要做的就是实现上述我们构想的过程，由于数据量大、计算复杂、计算次数多，所以我们选择使用计算机编程解决该问题。

### 2.1 环境准备
在开始正式代码前，我们应该先导入程序运行所必要的库，它们分别承担着数学运算、表格处理、尝试学习、神经网络等不同的职能！

In [ ]:
# 导入必要的库
import pandas as pd  # 用来处理表格数据的工具。我们可以把它想象成 Python 里的 Excel。
import numpy as np  # 用来做数学运算，比如加法、平均值。
import torch # 深度学习框架，提供张量计算和神经网络构建
import torch.nn as nn # 包含神经网络层、损失函数等核心组件。
import torch.optim as optim # 提供各种优化算法用于模型训练。

print('所有必要工具已导入')

### 2.2 创建线性回归模型🧩

现在我们来创建线性回归计算的重要模型：LinearRegressionModel，我们将在后续使用到该重要模型。

`LinearRegressionModel`的本质属性就是线性公式: $z = p \bullet x + q \bullet y$。我们选择使用它，正是因为原始数列的规律 $b_{n + 3} = p \bullet b_{n + 1} - q \bullet b_{n}$也是线性的，两个公式的形式完全一致。它是主要作用是后续输入自变量(x, y)，按照线性规则计算出预测值z，以待后续与`real_z`进行误差分析。

In [ ]:
# 定义线性回归模型
class LinearRegressionModel(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LinearRegressionModel, self).__init__()
        self.linear = nn.Linear(input_dim, output_dim, bias=False)
    
    def forward(self, x):
        out = self.linear(x)
        return out
print('LinearRegressionModel已创建成功')

**重要说明：**  
正因为数学模型与实际问题完美匹配，机器学习才能有效地找到正确的p和q值

### 2.3 数据读取与标准化🚀

在开始训练前，我们应该先导入数据并对数据进行简单处理:
1.  首先读取训练集CSV文件
2.  从data数据框中提取出 a_seq 列
4.  在训练参数前要对数据进行标准化处理：(原数据 - 平均值) / 标准差
5.  如果有必要，我们可以在相应位置，输出（print）数据，查看其内容

In [ ]:
# 数据读取与预处理
# 动手练习1，请按提示在相应位置补全代码

data =  pd.read_csv('a_seq_train.csv')  #读取训练集
a_seq = data['a_seq'].values #提取a_seq列
print('数据导入成功：')
print(a_seq)

# 对数据进行标准化
mean = np.mean(a_seq)  # 求平均值
std = np.std(a_seq)  # 求标准差

# ✅请在下方空白处补全代码：------------------------------------------------------------
a_seq =               # (原数据 - 平均值) / 标准差
print('数据标准化成功：')
print(a_seq)

### 2.4 为什么要进行数据标准化？🤔

上一小节我们提到了对数据进行标准化，现在让我们想一下什么是标准化，以及为什么要使用标准化后的数据，而不是原始数据。  
**🧮**想象一下，如果原始数据很大，比如：

$$128,\ 142,\ 135,\ 156,\ 121$$

直接使用这些数据计算时：

- 数值较大，计算机计算起来比较困难。
- 训练过程会变得很慢，而且结果可能不准确
- 就像我们用很大的数字做计算时容易出错一样

**让我们通过例子来理解标准化**：

就用这5个数据：$128,\ 142,\ 135,\ 156,\ 121$

计算过程：

1.  **平均值** = (128 + 142 + 135 + 156 + 121) ÷ 5 = **136.4**
2.  **标准差** ≈ 13.2（描述数据波动大小的值，有固定公式计算，这里我们直接使用结果）

    - 第一个数：(128 - 136.4) ÷ 13.2 ≈ -0.64
    - 第二个数：(142 - 136.4) ÷ 13.2 ≈ 0.42
    - 第三个数：(135 - 136.4) ÷ 13.2 ≈ -0.11
    - 第四个数：(156 - 136.4) ÷ 13.2 ≈ 1.48
    - 第五个数：(121 - 136.4) ÷ 13.2 ≈ -1.17

标准化后的数据：$- 0.64,0.42, - 0.11,1.48, - 1.17$

**✅ 填空（请填写在纸质材料上）：**  
标准化后，原本较大的数据变成了大小合适的数值，集中在 _______ 附近。

### 2.5 构建训练样本


所谓“构建训练样本”就是将现在已有的数据，按之前约定好的线性原则，用 aₙ 和 aₙ₊₁ 作为输入，用 aₙ₊₃ 作为预测目标，构建出x, y, real_z，并将它们设置成pytorch所需要的格式。

**代码执行说明**：

1.  `x` 对应递推关系中的 aₙ（第一个自变量）
2.  `y` 对应递推关系中的 aₙ₊₁（第二个自变量）
3.  `real_z` 对应递推关系中的 aₙ₊₃


In [ ]:
# 构建训练样本
x = np.array(a_seq[0:len(a_seq)-3])
y = np.array(a_seq[1:len(a_seq)-2])
real_z = np.array(a_seq[3:len(a_seq)])

print('训练样本已构建完毕')

### 2.6 准备模型输入数据

现在我们将数据转换为模型需要的格式，这是由于torch这个机器学习库的要求，在本步骤中，只对数据进行形状的改变而不对数据内容进行更改。

**代码执行说明**：

1.  `xy`是将自变量x和y放在一起的二元自变量，包含所有的aₙ和aₙ₊₁组合
2.  `real_z`是真实目标值，包含所有的aₙ₊₃
3.  后续模型将用`xy`计算出预测的z值，然后与`real_z`比较计算误差

In [ ]:
# 准备模型输入数据
# 按照torch库的要求将数据设置为其所需的形状：
xy = torch.tensor(np.vstack((x, y)).T, dtype=torch.float32)
real_z = torch.tensor(real_z, dtype=torch.float32).view(-1, 1)
print(xy)
print(real_z)

**✅ 填空（请填写在纸质材料上）：**  
`xy`是 _______（自/因）变量；`real_z`是真实值，它后续将会与预测的结果z进行误差分析。

## 第三部分：编程实践 - 模型训练与结果

### 3.1 创建线性回归模型🧩

现在我们来创建机器学习模型！

**代码执行说明**：

1.  `input_dim = 2` 表示模型有2个输入特征（即两个因变量x和y，亦是aₙ和aₙ₊₁）
2.  `output_dim = 1` 表示模型将输出1个预测值
3.  创建`LinearRegressionModel`模型并命名为`model`，它的主要作用在2.2小节有详细介绍
4.  `criterion` 是损失函数计算器（可理解为误差计算器），可计算均方误差；`optimizer` 是优化器，负责根据误差自动调整模型参数p和q

In [ ]:
# 创建线性回归模型
# 动手练习2，请按提示在相应位置补全代码

# ✅请在下方两行空白处补全代码：------------------------------------------------------------
input_dim =   # 输入维度，即输入几元？
output_dim =   # 输出维度，即输出几元？
model = LinearRegressionModel(input_dim, output_dim)

# 设置训练参数
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
print('线性回归模型model已创建！')

**✅ 填空（请填写在纸质材料上）：**   
我们的模型有 _______ 个输入特征，输出 _______ 个预测值。

### 3.2 模型训练循环🔄

现在开始模型的训练！

**代码执行说明**：

1.  `num_epochs = 10000` 表示模型要练习10000次
2.  `z = model(xy)`：将我们之前准备好的二元自变量xy传入model（回忆：xy包含所有的aₙ和aₙ₊₁组合），model会根据当前p和q的值计算出预测值z
3.  `loss = criterion(z, real_z)`：将预测值z与真实值real_z进行比较，计算损失函数
4.  `optimizer.zero_grad(); loss.backward(); optimizer.step()`：根据损失函数结果，智能调整p和q的值，调整后的p和q会使预测值z更接近真实值real_z。
5.  通过取余和逻辑判断语句，定期显示当前训练效果


In [ ]:
# 模型训练循环
# 动手练习3，请按提示在相应位置补全代码

# 设置模型训练的次数，一般可根据项目需要，定为5000、10000、100000等
# ✅请在下方空白处补全代码：------------------------------------------------------------
num_epochs = 

# 循环开始
# ✅请在下方括号处补全代码：------------------------------------------------------------
for epoch in range(      ):
    # 输入二元自变量xy，计算预测值
    z = model(xy)
    
    # 计算损失
    # ✅请在下方括号处补全代码：------------------------------------------------------------
    loss = criterion(      )  # 计算z与real_z之间的损失
    
    # 调整p和q参数
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    # 每100次打印一次损失函数及当前p和q的训练结果
    if epoch % 100 == 0:
        print(epoch, '//', num_epochs)
        print(f'Loss: {loss.item():.4f}')
        print(model.state_dict())
# 循环结束

**✅ 填空（请填写在纸质材料上）：**  
循环语句中的 _______ 变量控制着模型要训练的总次数。

### 3.3 提取训练结果🎯

10000次训练完成后，我们需要提取找到的p和q值，下面三行代码是在从model中获取模型训练后学到的参数p和q

In [ ]:
# 提取训练结果，获取模型的状态
state_dict = model.state_dict()
p_value = state_dict['linear.weight'].tolist()[0][1]
q_value = - state_dict['linear.weight'].tolist()[0][0]

print('恭喜你终于提取到了a_seq_train数据集的参数p与q')
print(p_value, q_value)

**🎉恭喜你终于提取到了a_seq_train数据集的参数p与q**

## 第四部分 处理三个数据集

回到主程序，我们用同样的方法处理另外两个数据集：验证集、测试集，只需在读取初始数据时，将路径分为改为对应数据集的路径即可。

In [ ]:
# 处理其他数据集，注意，此处仅展示流程，不需要进行填空或补充代码
# 读取验证集
data =  pd.read_csv('a_seq_validation.csv')
#  。。。。。。
# 后续需要重复上述所有代码

# 读取测试集
data =  pd.read_csv('a_seq_test.csv')
#  。。。。。。
# 后续需要重复上述所有代码

**小思考**

如果我们需要一次性计算三个数据集的p,q值，该如何处理呢？要重复写三次计算代码吗？

我们发现在处理三次数据集的过程中，有些核心代码是一直不变、重复使用的，一个好的想法是将上述会重复使用的核心代码嵌套进一个【函数】内，当每次需要使用核心代码时，只需要调用一次该即可，我们可以为该函数起名：pq_regression.在下方编写一下该函数吧！


In [ ]:
# 动手练习4，请仔细思考核心代码应该是上述学过的哪些内容

# 定义函数
def pq_regression(data):
    # ✅请在下方空白处复制粘贴核心代码：------------------------------------------------------------



    
    
    return p_value, q_value

# 主程序
# 1. 读取三个数据集
data_train = pd.read_csv("a_seq_train.csv")
data_validation = pd.read_csv("a_seq_validation.csv")
data_test = pd.read_csv("a_seq_test.csv")

# 2. 分别计算每个数据集的p和q
p_train, q_train = pq_regression(data_train)
p_validation, q_validation = pq_regression(data_validation)
p_test, q_test = pq_regression(data_test)

# 3. 整理并输出结果
p = [p_train, p_validation, p_test]
q = [q_train, q_validation, q_test]
df_params = pd.DataFrame({'p': p, 'q': q})
print(df_params)

## 第五部分：完整流程与学习总结

### 5.1 完整程序流程回顾📋

让我们回顾整个程序的完整执行流程：

1.  读取三个CSV文件，获得三个数据集
2.  对每个数据集分别调用 pq_regression 函数
3.  每个函数调用都会完成：数据标准化 → 构建样本 → 模型训练 → 提取参数
4.  最终得到三组p、q值并保存

In [ ]:
# 主程序开始
# 1. 读取三个数据集
data_train = pd.read_csv("a_seq_train.csv")
data_validation = pd.read_csv("a_seq_validation.csv")
data_test = pd.read_csv("a_seq_test.csv")

# 2. 分别计算每个数据集的p和q
p_train, q_train = pq_regression(data_train)
p_validation, q_validation = pq_regression(data_validation)
p_test, q_test = pq_regression(data_test)

# 3. 整理并输出结果
p = [p_train, p_validation, p_test]
q = [q_train, q_validation, q_test]
df_params = pd.DataFrame({'p': p, 'q': q})
print(df_params)

# 4. 保存结果
df_params.to_csv('your_name_submission.csv', index=False)

### 5.2 学习收获总结🏆

通过这个完整的学习过程，我们实现了：

✅ **问题理解**：从数学规律到实际挑战的转化  
✅ **思路建立**：从传统数学到 _______ 的思维转变  
✅ **编程实践**：从理论到代码的完整实现  
✅ **结果验证**：三个数据集的参数求解和验证

**核心收获**：

- 理解了线性回归的基本原理
- 掌握了机器学习解决实际问题的完整流程
- 体验了从数学问题到编程实现的转化过程
- 感受了智能算法自动寻找规律的神奇魅力

### 5.3 扩展思考💭

1.  如果数据量很少（比如只有10个数据），训练效果会 _______？
2.  如果不对数据进行**标准化**会有什么问题？
3.  为什么线性回归能解决这个数列问题？
4.  如果规律不是**线性**的，这个方法还适用吗？

**✅ 填空（请填写在纸质材料上）：**  
线性回归能解决这个数列问题，是因为原始数列的规律是 _______ 的。

**🎉恭喜你完成了这次精彩的机器学习探索之旅！**

你已经掌握了：

- 数学问题建模技巧
- 机器学习的基本思想
- 线性回归的核心原理
- 编程实现能力

**🚀 继续探索数学与计算机的奇妙世界吧！**

----
© 2025 付佳兴、马衍龙、郑子杰及AI4S TEEN CAMP组委会  
本教学材料仅供课堂教学与学术交流使用，未经书面许可，不得用于任何商业用途。